In [1]:
import numpy as np 
import scipy as sp  
import random     
import bokeh 
import csv
import pandas as pd 
import struct
import os, pickle,sys


In [2]:
with open('../data/5415172555729652564.pkl', 'rb') as pkl_file:
    df_list = pickle.load(pkl_file)

In [3]:
df = pd.concat(df_list)

In [4]:
#df = pd.read_csv('long.csv')
#sf = pd.read_csv('short.csv')
#d1 = pd.read_csv('test1.csv')
#d2 = pd.read_csv('test2.csv')
#d3 = pd.read_csv('test3.csv') #This is the one I have notes for 
#d4 = pd.read_csv('test4.csv') #Driving Home

#s1 = pd.read_csv('round1.csv')
#s2 = pd.read_csv('round2.csv')
#df = pd.read_csv('round3.csv')
#df = pd.read_csv('20170427-stationary-2-leith-croydon.csv')
df.Timestamp = df.Timestamp - min(df['Timestamp'])


In [5]:
df.columns

Index([u'index', u'file_index', u'ObjectId', u'Flags',
       u'trackedByStationaryModel', u'mobile', u'motionModelValidated',
       u'ObjectAge', u'Timestamp', u'ObjectPredAge', u'Classification',
       u'ClassCertainty', u'ClassAge', u'ObjBoxCenter_X', u'ObjBoxCenter_Y',
       u'ObjBoxCenterSigma_X', u'ObjBoxCenterSigma_Y', u'ObjBoxSize_X',
       u'ObjBoxSize_Y', u'ObjCourseAngle', u'ObjCourseAngleSigma',
       u'ObjBoxOrientation', u'ObjBoxOrientationSigma', u'RelVelocity_X',
       u'RelVelocity_Y', u'RelVelocitySigma_X', u'RelVelocitySigma_Y',
       u'AbsVelocity_X', u'AbsVelocity_Y', u'AbsVelocitySigma_X',
       u'AbsVelocitySigma_Y', u'RefPointLocation', u'RefPointCoords_X',
       u'RefPointCoords_Y', u'RefPointCoordsSigma_X', u'RefPointCoordsSigma_Y',
       u'RefPointPosCorrCoeffs', u'ObjPriority', u'ObjExtMeasurement',
       u'EgoLatitude', u'EgoLongitude', u'EgoAltitude', u'EgoHeadingRad',
       u'EgoPosTimestamp', u'GPSFixStatus', u'ObjPrediction', u'Object_X',
  

In [6]:
#motionModelValidated is always true

df.trackedByStationaryModel = (df.trackedByStationaryModel==1)
df.mobile = (df.mobile==1)
df['ObjPrediction'] = (df.ObjectPredAge > 0)

In [7]:


from matplotlib import pyplot as plt
Nlines = 260 
from itertools import permutations     
color_lvl = 8     
rgb = np.array(list(permutations(range(0,256,color_lvl),3))) 
rgb_m = np.array(list(permutations(range(0,256,color_lvl),3)))/255.0

from random import sample
colors = sample(rgb,Nlines) 
colors_m = sample(rgb_m,Nlines)

#struct.pack('BBB',*rgb).encode('hex')

#df['color'] = [tuple(colors[i]) for i in df.ObjectId]
df['color'] = ["#" + struct.pack('BBB',*colors[i]).encode('hex') for i in df.ObjectId]
df['color_m'] = [tuple(colors_m[i]) for i in df.ObjectId]

In [8]:
#fill_color= [df['color'].iloc[i] if df['mobile'].iloc[i] else None for i in range(len(df))]

In [9]:
#fill_color= [df['color'].iloc[i] if df['mobile'].iloc[i] else None for i in range(len(df))]
fill_color= [color if flag else None for color, flag in zip(df['color'],df['mobile'])]


In [10]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
df.columns

Index([u'index', u'file_index', u'ObjectId', u'Flags',
       u'trackedByStationaryModel', u'mobile', u'motionModelValidated',
       u'ObjectAge', u'Timestamp', u'ObjectPredAge', u'Classification',
       u'ClassCertainty', u'ClassAge', u'ObjBoxCenter_X', u'ObjBoxCenter_Y',
       u'ObjBoxCenterSigma_X', u'ObjBoxCenterSigma_Y', u'ObjBoxSize_X',
       u'ObjBoxSize_Y', u'ObjCourseAngle', u'ObjCourseAngleSigma',
       u'ObjBoxOrientation', u'ObjBoxOrientationSigma', u'RelVelocity_X',
       u'RelVelocity_Y', u'RelVelocitySigma_X', u'RelVelocitySigma_Y',
       u'AbsVelocity_X', u'AbsVelocity_Y', u'AbsVelocitySigma_X',
       u'AbsVelocitySigma_Y', u'RefPointLocation', u'RefPointCoords_X',
       u'RefPointCoords_Y', u'RefPointCoordsSigma_X', u'RefPointCoordsSigma_Y',
       u'RefPointPosCorrCoeffs', u'ObjPriority', u'ObjExtMeasurement',
       u'EgoLatitude', u'EgoLongitude', u'EgoAltitude', u'EgoHeadingRad',
       u'EgoPosTimestamp', u'GPSFixStatus', u'ObjPrediction', u'Object_X',
  

In [12]:
single_track = df[df['uniqueId']==13048]
#print single_track
p = figure(plot_height=600, plot_width=700, title="",x_range=(-40,20),y_range=(-30,30))
#p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
#p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
p.image_url(['leith-croydon.png'], x=-15.275, y=-3.1, w=147.45, h=77.0, angle=0, anchor='center', global_alpha=0.7)
p.circle(x=single_track.Object_X, y=single_track.Object_Y, size=4, fill_color="green", line_color="green")
show(p,notebook_handle=True)

In [14]:
record_dict = {}
df.origin.unique()
for origin in df.origin.unique():
    for destination in df.destination.unique():
        if origin in destination:
            continue
        dict_key = str(origin + '-' + destination)
        print origin + '-' + destination
        class_df = df[(df['origin']==origin) & (df['destination']==destination)]
        class_df = class_df.sort_values('distance',ascending=False)
        max_dis = int(max(class_df.distance.unique())) # max_dis = 60 # 
        min_dis = int(min(class_df.distance.unique())) #min_dis = -10 #
        n_vehicles = len(class_df.uniqueId.unique())
        vehicle_cnt = 0
        records = []
        for uniqueId in class_df.uniqueId.unique():
            vehicle_cnt+=1
            sys.stdout.write("\rWorking on vehicle %d of %d" % (vehicle_cnt, n_vehicles))
            sys.stdout.flush()
            vehicle_df = class_df[class_df['uniqueId']==uniqueId]
            vehicle_df = vehicle_df.sort_values('distance',ascending=False)
            for d_thresh in np.arange(min_dis,max_dis,0.5):
                #print "distance: " + str(d_thresh)
                
                #Find the closest but not over point, add to a thing.
                tp = vehicle_df[vehicle_df['distance']<d_thresh]
                try:
                    record = tp.iloc[range(1)] #
                except IndexError:
                    # This track does not exist at said distance. Continue
                    continue
                record = record.assign(d_thresh=np.repeat(d_thresh, len(record)))
                # if it is not within 1 meter, there is no data, and we are redundantly appending the last results
                if record.iloc[0].distance > (d_thresh -1):
                    records.append(record)
        print "Concatenating Records..."
        records = pd.concat(records)
        print "done"
        record_dict[dict_key] = records
        

east-north
Working on vehicle 2959 of 2959Concatenating Records...
done
east-south
Working on vehicle 449 of 449Concatenating Records...
done
north-south
Working on vehicle 557 of 557Concatenating Records...
done
north-east
Working on vehicle 2499 of 2499Concatenating Records...
done
south-north
Working on vehicle 749 of 749Concatenating Records...
done
south-east
Working on vehicle 1222 of 1222Concatenating Records...
done


In [15]:
track_classes = list(record_dict.keys())
track_colors = ['Cyan','DarkBlue','Chartreuse', 'FireBrick','Fuchsia','SeaGreen']
origin_destination = 'east-north'
speeds_dt = {}
headings_dt = {}
distances_dt = {}
population_dt = {}
for track_class, color in zip(track_classes,track_colors):
    tf = record_dict[track_class]
    distances = np.sort(tf['d_thresh'].unique())
    distances_dt[track_class] = distances
    speeds = []
    headings = []
    track_pop = []
    for d_thresh in distances:
        df_at_dis = tf[tf['d_thresh']==d_thresh]
        #print df_at_dis.columns
        #Collect various data for plot here. I am going to do speed and heading
        speeds.append(np.average(df_at_dis['AbsVelocity'])*3.6)
        headings.append(np.average(df_at_dis['ObjBoxOrientation']))
        track_pop.append(len(df_at_dis))
        
    speeds_dt[track_class] = speeds
    headings_dt[track_class] = headings
    population_dt[track_class] = track_pop

        

In [16]:
#PLOT STUFF FROM record_dict
output_notebook()
p = figure(plot_height=600, plot_width=700, title="Speeds")
for track_class, color in zip(track_classes,track_colors):
    p.line(x=distances_dt[track_class], y=speeds_dt[track_class], line_width=4, line_color=color)
show(p,notebook_handle=True)


Loading BokehJS ...

In [17]:
output_notebook()
p = figure(plot_height=600, plot_width=700, title="Headings")
for track_class, color in zip(track_classes,track_colors):
    p.line(x=distances_dt[track_class], y=headings_dt[track_class], line_width=4, line_color=color)
show(p,notebook_handle=True)

Loading BokehJS ...

In [18]:
output_notebook()
p = figure(plot_height=600, plot_width=700, title="Population")
for track_class, color in zip(track_classes,track_colors):
    p.line(x=distances_dt[track_class], y=population_dt[track_class], line_width=4, line_color=color)
show(p,notebook_handle=True)

Loading BokehJS ...

In [19]:
list(record_dict.keys())

['east-south',
 'east-north',
 'south-east',
 'south-north',
 'north-east',
 'north-south']

In [20]:
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, push_notebook
from ipywidgets import interact
import ipywidgets

output_notebook()

time_max = max(df['Timestamp'])
time_min = min(df['Timestamp'])

time_slider = Slider(start=time_min,end=time_max,value=time_min,step=1,title="Time")
source_rect = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_circle = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[],RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_crossH = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_crossW = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_ref_pt = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg_accel = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
#Sliders:
slider_time = ipywidgets.FloatSlider(
    value = (time_max+time_min)/2, min=time_min, max=time_max, step = 0.1, description="Time",slider_color="red")
window_slider = ipywidgets.FloatSlider(
    value = 5, min=1, max=time_max, step = 0.1, description="Time Window",slider_color="blue")
min_conf_slider = ipywidgets.FloatSlider(
    value = 5, min=0, max=100, step = 1, description="Min Classification Score",slider_color="green")


hover = HoverTool(tooltips=[
    ("ObjectId", "@ObjectId"),
    ("RefPointLocation", "@RefPointLocation"),
    #("$", "@revenue")
])

p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-40,20),y_range=(-30,30))
#p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
#p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
p.image_url(['leith-croydon.png'], x=-15.275, y=-3.1, w=147.45, h=77.0, angle=0, anchor='center', global_alpha=0.7)
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossH, fill_color="color",
       line_color="color")
p.circle(x="x", y="y", source=source_circle, size=4, fill_color="fill_color", line_color="line_color")
p.circle(x="x", y="y", source=source_ref_pt, size=3, fill_color="black", line_color="black")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_rect, fill_color=None,
       line_color="color")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossW, fill_color="color",
       line_color="color")
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='orange',source=source_seg)
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='pink',source=source_seg_accel)


show(p,notebook_handle=True)

def select_data(time_val, time_window, min_conf):
    #time_val = time_slider.value
    #t_min = time_val-(float(time_window)/2.0)
    #t_max = time_val+(float(time_window)/2.0)
    t_min = time_val-time_window
    t_max = time_val

    selected = df[
        (df.Timestamp > t_min)
        &
        (df.Timestamp < t_max)
        &
        (df.Classification >3 )
        &
        (df.ClassCertainty > min_conf)
    ]
    
    return_df = pd.DataFrame()
    
    for obj_id in selected.ObjectId.unique():
        object_df = selected[selected.ObjectId==obj_id].sort_values(['Timestamp'],ascending=False)
        selected_record=object_df.iloc[0].copy()
        if len(object_df) > 1:
            selected_record['AbsAccel_X'] = object_df['AbsVelocity_X'].iloc[0] - object_df['AbsVelocity_X'].iloc[1]
            selected_record['AbsAccel_Y'] = object_df['AbsVelocity_Y'].iloc[0] - object_df['AbsVelocity_Y'].iloc[1]
        else:
            selected_record['AbsAccel_X'] = 0
            selected_record['AbsAccel_Y'] = 0
        return_df = return_df.append(selected_record)
        
    # Only draw box if car was seen in last 0.5 seconds. 
    # I really only want if it was seen at THIS timestep, but this is close enough
    recent_only_df = return_df[return_df['Timestamp']> (time_val-0.2)]
    return recent_only_df, selected #selected

def update(time_value=slider_time,time_window=window_slider,min_conf=min_conf_slider):
    df_rect, df_circle = select_data(time_value,time_window,min_conf)
    #print df_rect
    source_rect.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxSize_X'],
        height=df_rect['ObjBoxSize_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_ref_pt.data = dict(
        x=df_rect['RefPointCoords_X'],
        y=df_rect['RefPointCoords_Y'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(df_rect['AbsVelocity_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(df_rect['AbsVelocity_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg_accel.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(10*df_rect['AbsAccel_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(10*df_rect['AbsAccel_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    source_circle.data = dict(
        x=df_circle['ObjBoxCenter_X'],
        y=df_circle['ObjBoxCenter_Y'],
        fill_color= [color if not flag else "#FFFFFF" for color,flag in zip(df_circle['color'],df_circle['ObjPrediction'])],
        line_color=df_circle['color'],
        ObjectId=df_circle['ObjectId'],
        #RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_circle['Timestamp']
    )
    source_crossW.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxCenterSigma_X'],
        height=[0.1]*len(df_rect),
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    #'ObjBoxCenterSigma_Y'
    source_crossH.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=[0.1]*len(df_rect),
        height=df_rect['ObjBoxCenterSigma_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        Timestamp=df_rect['Timestamp']
    )
    
    push_notebook()


update(time_max,1,5)  # initial load of the data



Loading BokehJS ...

In [21]:
#If this fails run $ jupyter nbextension enable --py --sys-prefix widgetsnbextension

interact(update,time_val=slider_time,time_window=window_slider,min_conf=min_conf_slider)


<function __main__.update>